In [1]:
import pandas as pd
import numpy as np
from restructuration import restructuration_noeuds_realise as rest_node_def

from multiprocessing import Pool
from tqdm import tqdm
import os, re

import networkx as nx
from itertools import chain
from collections import Counter

In [2]:
path = '/home/administrateur/Bureau/Datagrosyst/data_entrepot_outils/'
donnees = dict(
    noeuds_realise = pd.read_csv(path + 'noeuds_realise.csv'),
    zone = pd.read_csv(path + 'zone.csv'),
    parcelle = pd.read_csv(path + 'parcelle.csv'),
    sdc = pd.read_csv(path + 'sdc.csv'),
    dispositif = pd.read_csv(path + 'dispositif.csv'),
    domaine = pd.read_csv(path + 'domaine.csv'),
    intervention_realise = pd.read_csv(path + 'intervention_realise.csv'),
    typoculture = pd.read_csv(path + 'typologie_can_culture.csv'),
    culture = pd.read_csv(path + 'culture.csv')
)

/tmp/ipykernel_333004/2102657812.py:9: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  intervention_realise = pd.read_csv(path + 'intervention_realise.csv'),


In [3]:
intervention_realise = donnees['intervention_realise'][['id','type','noeuds_realise_id','combinaison_outil_id']].rename(columns={
    'id':'intervention_realise_id',
    'type':'intervention_type',
    'noeuds_realise_id':'noeud_realise_id'})
node = donnees['noeuds_realise'].rename(columns={
    'id':'noeud_realise_id'})
typoculture = donnees['typoculture'][['culture_id','typocan_culture_sans_compagne']]
culture = donnees['culture'][['id','nom']].rename(columns={
    'id':'culture_id'})
zone = donnees['zone'][['id','code','nom','campagne','surface','parcelle_id']].rename(columns={
    'id':'zone_id',
    'code':'zone_code',
    'nom':'zone_nom',
    'surface':'zone_surface'})
parcelle = donnees['parcelle'][['id','code','nom','surface','sdc_id','domaine_id','commune_id']].rename(
    columns={ # ajout 'campagne' ?
    'id':'parcelle_id',
    'code':'parcelle_code',
    'nom':'parcelle_nom',
    'surface':'parcelle_surface',
    'commune_id': 'commune_id_from_parcelle',
    'domaine_id' : 'domaine_id_from_parcelle'})
sdc = donnees['sdc'][['id','code','nom','code_dephy','reseaux_ir','dispositif_id']].rename(columns={ # ajout 'campagne' ?
    'id':'sdc_id',
    'code':'sdc_code',
    'nom':'sdc_nom'})
dispositif = donnees['dispositif'][['id','code','type','domaine_id']].rename(columns={
    'id':'dispositif_id',
    'code':'dispositif_code',
    'type':'dispositif_type'})
domaine = donnees['domaine'][['id','nom','code','commune_id']].rename(columns={
    'id':'domaine_id',
    'code':'domaine_code',
    'nom' : 'domaine_nom'})
# rest_node = rest_node_def(donnees)
# rest_node = rest_node.rename(columns={'id':'noeud_realise_id'})

In [4]:
df = intervention_realise.merge(node, on='noeud_realise_id', how='outer') # On fait un outter car on veux garder les cultures sans interv
df = df.merge(zone, on='zone_id', how='outer') # Si on commence par node et que l'on fait un left, cela signifie que l'on prend que les zone contenant une culture ! La on fait un outer pour toute les avoir
df = df.merge(parcelle, on='parcelle_id', how='left')
df = df.merge(sdc, on='sdc_id', how='left')
df = df.merge(dispositif, on='dispositif_id', how='left')
df['domaine_id'] = df['domaine_id'].fillna(df['domaine_id_from_parcelle'])
df = df.merge(domaine, on='domaine_id', how='left')

In [5]:
dfparc_w_c = node.merge(zone, on='zone_id', how='left') # Si on commence par node et que l'on fait un left, cela signifie que l'on prend que les zone contenant une culture !

print("Pourcentage de parcelles qui n'ont aucune culture sur aucune zone: ",
      round((len(set(parcelle.parcelle_id)) - len(set(dfparc_w_c.parcelle_id))) / len(set(parcelle.parcelle_id)) * 100, 2), '% des', len(set(parcelle.parcelle_id)), 'parcelle_id différents')

Pourcentage de parcelles qui n'ont aucune culture sur aucune zone:  41.9 % des 236629 parcelle_id différents


In [6]:
# voir quelles sont les cultures sur les zones sans interventions pour voir si on garde ou pas les zones sans interventions

list_zone_with_zero_interv = list(node.loc[~(node['noeud_realise_id'].isin(set(donnees['intervention_realise']['noeuds_realise_id']))), 'zone_id'].unique())
psansc = df.loc[df['zone_id'].isin(list_zone_with_zero_interv)]
psansc = psansc.merge(culture, on='culture_id', how='left')
psansc = psansc.merge(typoculture, on='culture_id', how='left')

# Les typologies de cultures sans interventions :
display(psansc.groupby('typocan_culture_sans_compagne')['typocan_culture_sans_compagne'].count().sort_values(ascending=False))
# Parmis les cultures sans typologies :
display(psansc.loc[psansc['typocan_culture_sans_compagne'].isin(['NoLink-sp-crop','NoInput-sp','Autre'])].groupby('nom')['nom'].count().sort_values(ascending=False))

typocan_culture_sans_compagne
Prairie temporaire                      8639
Autre                                   6783
Céréales à paille hiver                 2513
Maïs                                    2098
Légume                                  1198
NoInput-sp                               701
Mélange fourrager                        537
Protéagineux                             522
Colza                                    368
Céréales à paille printemps              268
Culture ornementale                      255
Tournesol                                223
Pomme de terre                           127
Betterave                                 98
Plante aromatique ou médicinale           79
NoLink-sp-crop                            44
Fraisier                                  31
Lin                                       30
Oléagineux (hors Colza et Tournesol)      27
Pommier                                   21
Prunier                                   16
Vigne                    

nom
Espèces diverses     6006
Mélange               153
Jachère               152
Autres Potagères       80
Relais seules          66
                     ... 
Blé / Tournesol         1
patates douces          1
songe                   1
phacelie                1
plante de service       1
Name: nom, Length: 179, dtype: int64

In [7]:
# On controle les numéros dephy de df, il faut qu'ils fasse exactement 3 lettre + 5 chiffres
# Pattern exact : 3 lettres puis 5 chiffres
pattern = r'^[A-Z]{3}(?:[0-9]|X)\d{4}$'
mask_vide = ((df['code_dephy'].isnull()) | (df['code_dephy']=='NAN') | (df['code_dephy']=='-')  | (df['code_dephy']=='9999'))

df_num_dephy_vide = df[mask_vide][['parcelle_id','sdc_id','code_dephy']]
print(f"Avant nettoyage : {len(df_num_dephy_vide)} parcelles sans numéro dephy")

df_invalid_num_dephy = df[(~df['code_dephy'].str.fullmatch(pattern, na=False)) & (~mask_vide)][['sdc_id', 'code_dephy']]
print(f"Avant nettoyage : {len(df_invalid_num_dephy)} parcelles avec un numéro dephy en dehors du pattern")

# Nettoyage des espaces et tabulations
df['code_dephy'] = df['code_dephy'].astype(str).\
    str.replace('PPZ', '').\
        str.replace('_', '').\
            str.replace(' ', '').\
                str.replace('\t', '').\
                    str.upper()

print("")
print(f"Après nettoyage : {len(
    df[(~df['code_dephy'].str.fullmatch(pattern, na=False)) & (~mask_vide)][['sdc_id', 'code_dephy']]
    )} parcelles avec un numéro dephy en dehors du pattern (hors numero dephy vides)")

Avant nettoyage : 272332 parcelles sans numéro dephy
Avant nettoyage : 15911 parcelles avec un numéro dephy en dehors du pattern

Après nettoyage : 2964 parcelles avec un numéro dephy en dehors du pattern (hors numero dephy vides)


In [8]:
# A quoi ressemble les patterns pas bons ?
set(df[(~df['code_dephy'].str.fullmatch(pattern, na=False)) & (~mask_vide)]['code_dephy'])

{'0',
 '70CI53GC',
 'ARENSEIGNER',
 'ARF26542MELROSE',
 'ARF26542PINK',
 'ARF26617REF',
 'ARF600021',
 'CTF1762',
 'ESSAI',
 'GCF345272',
 'GCF4EXEMPLE00',
 'GFCXXJOU',
 'LGFEARLFONTANELLE',
 'PY27671',
 'SDC1JOUAN',
 'SDC2FABRICE',
 'VIF3432'}

In [9]:
# On groupe par parcelle pour avoir le nombre d'intervention par parcelle et le nombre de zone
cols_for_first = [col for col in df.columns if col not in [
    'parcelle_id',

    'intervention_realise_id',
    'noeud_realise_id',
    'zone_id',
    'combinaison_outil_id',
    'intervention_type',
    'rang',
    'culture_id',
    'nom',
    'culture_id',
    'zone_code',
    'zone_nom',
    'zone_surface'
    ]]


contains_RS = lambda x: (
    'recolte_et_semis'
    if (x.astype(str).str.contains('RECOLTE', na=False).any()
        and x.astype(str).str.contains('SEMIS', na=False).any())
    else (
        'recolte_seulement'
        if x.astype(str).str.contains('RECOLTE', na=False).any()
        else (
            'semis_seulement'
            if x.astype(str).str.contains('SEMIS', na=False).any()
            else 'manque_recolte_et_semis'
        )
    )
)

has_value = lambda x: 'oui' if x.notnull().any() else 'non'

df_parcelle = df.groupby('parcelle_id').agg(
    {
        **{c: 'first' for c in cols_for_first},
        **{c: 'nunique' for c in ['zone_id','noeud_realise_id','intervention_realise_id']},
        'intervention_type': contains_RS,
        'combinaison_outil_id': has_value
    }
).reset_index()

In [10]:
# On groupe par nom et surface de parcelle + meme numero DEPHY !

grouped_dephy = df_parcelle.groupby(['parcelle_nom','parcelle_surface','code_dephy']).apply(lambda x: pd.Series({
    'campagne': list(set(x['campagne'])),
    'parcelle_code' : list(set(x['parcelle_code'])),
    'parcelle_code_nb': len(list(set(x['parcelle_code']))),
    'parcelle_id': list(set(x['parcelle_id'])),
    'p_campagne_dbl':   [z for z in 
                        list(x.groupby('campagne').agg({'parcelle_id' : lambda y : list(set(y)) if len(set(y)) > 1 else None })['parcelle_id'])
                        if z is not None] ,
    'contient_R_S': list(set(x['intervention_type'])),
    'contient_outil': list(set(x['combinaison_outil_id'])),
    'zone_nb': list(x['zone_id']),
    'culture_nb': list(x['noeud_realise_id']),
    'interv_nb': list(x['intervention_realise_id']),
    'reseaux_ir': list(set(x['reseaux_ir'])),
    'nb_reseaux_ir': len(list(set(x['reseaux_ir']))),
    'sdc_code': list(set(x['sdc_code'])),
    'sdc_id': list(set(x['sdc_id'])),
    'sdc_code_nb': len(set(x['sdc_code'])),
    'dispositif_code': list(set(x['dispositif_code'])),
    'dispositif_id': list(set(x['dispositif_id'])),
    'dispositif_code_nb': len(set(x['dispositif_code'])),
    'domaine_code': list(set(x['domaine_code'])),
    'domaine_id': list(set(x['domaine_id'])),
    'domaine_code_nb': len(set(x['domaine_code'])),
    'commune_id': list(set(x['commune_id']))
    }), include_groups=False).reset_index().sort_values(by = 'parcelle_code_nb', ascending = False)

In [11]:
grouped_dephy.loc[grouped_dephy['parcelle_code_nb'] >= 2, 'parcelle_id'].sample(1, random_state=0).explode()

70244    fr.inra.agrosyst.api.entities.Plot_cfc1ed47-10...
70244    fr.inra.agrosyst.api.entities.Plot_d8b19b30-8e...
Name: parcelle_id, dtype: object

In [12]:
# # Récupere les reseau IR qui ont changer entre les parcelles détecté comme étant de meme code
# plsrs_res_ir = pd.DataFrame(columns = ['reseau_detect', 'sdc_id', 'reseaux_ir'])
# for res_ir in set(tuple(i) for i in grouped_dephy.loc[grouped_dephy['nb_reseaux_ir']>1, 'reseaux_ir']) :
#     print(res_ir)
#     temp = sdc.loc[sdc['sdc_id'].isin(grouped_dephy.loc[grouped_dephy['reseaux_ir'].apply(lambda x : tuple(x) == res_ir), 'sdc_id'].explode().unique()), ['sdc_id','reseaux_ir']]
#     temp['reseau_detect'] = '  &  '.join(str(val) for val in res_ir)
#     plsrs_res_ir = pd.concat([plsrs_res_ir, temp])

In [13]:
# # 202 numéro dephy qui ont des soucis de duplication de nom/surface de parcelles
# # princiapelement à cause de 2 IR (Tinoco et Cornet)
# dephy_test = grouped_dephy.loc[grouped_dephy['parcelle_code_nb'] > 1]

# dephy_test.loc[dephy_test.duplicated(subset=['parcelle_nom','parcelle_surface'], keep=False)].sort_values(by=['parcelle_nom','parcelle_surface'])

In [14]:
df_prolong = grouped_dephy.copy()

In [15]:
# # 12 frises détecté avec un chevauchement d'année sur les parcelles que l'on voudrait joindre entre elles
# df_prolong.loc[df_prolong['p_campagne_dbl'].apply(lambda x: len(x) > 0),]

In [16]:
# Détecté si un code donné à une frise est présent dans une autre frise
def detect_code_in_other_prolong(row, df_prolong):
    other_rows = df_prolong[df_prolong.index != row.name]
    codes_in_other_frise = set()
    index_in_other_frise = set()
    for code in row['parcelle_code']:
        matching_rows = other_rows[other_rows['parcelle_code'].apply(lambda x: code in x)]
        if not matching_rows.empty:
            codes_in_other_frise.add(code)
            index_in_other_frise.update(matching_rows.index.tolist())
    codes_in_other_frise = codes_in_other_frise if codes_in_other_frise else None
    index_in_other_frise = list(index_in_other_frise) if index_in_other_frise else None
    return codes_in_other_frise, index_in_other_frise

def parallel_apply(df, func, n_workers=4):
    with Pool(n_workers) as pool:
        results = pool.starmap(func, [(row, df) for _, row in df.iterrows()])
    return results

# Exemple d'utilisation
results = parallel_apply(df_prolong, detect_code_in_other_prolong)

# On sépare les résultats dans deux colonnes distinctes
df_prolong['code_in_other_prolong'] = [result[0] for result in results]
df_prolong['index_of_other_prolong'] = [result[1] for result in results]

In [17]:
df_prolong.to_csv('/home/administrateur/Bureau/groupement_parcelle_nom_surface_ndephy.csv', index=True)

In [18]:
G = nx.Graph()
for idx, row in df_prolong.iterrows():
    for o in row.get("index_of_other_prolong", []) or []:
        G.add_edge(idx, o)

composantes = list(nx.connected_components(G))

fusion = []
cols_exclues = ["index_of_other_prolong", "code_in_other_prolong"]
cols_exclues = cols_exclues + ['dispositif_code_nb','sdc_code_nb','domaine_code_nb','nb_reseaux_ir']

for comp in composantes:
    sub = df_prolong.loc[list(comp)]
    row = {"index_originaux": list(comp)}

    for col in df_prolong.columns:
        if col in cols_exclues: 
            continue

        vals = []
        for v in sub[col].dropna():
            vals.extend(v if isinstance(v, list) else [v])

        # Consolidation : aplatir p_campagne_dbl si liste de listes
        if col == "p_campagne_dbl" and any(isinstance(x, list) for x in vals):
            vals = list(chain.from_iterable(vals))

        if col == 'campagne':
            row[col] = vals  # garder les doublons pour 'campagne'
        else:
            if vals == []:
                row[col] = None
            else:
                row[col] = set(vals)

    # Détection des doublons dans "campagne"
    if "campagne" in row:
        c = Counter(row["campagne"])
        row["doublons_annees"] = {a for a, n in c.items() if n > 1} or None
    else:
        row["doublons_annees"] = None

    fusion.append(row)

df_fusion = pd.DataFrame(fusion)

In [19]:
df_fusion.to_csv('/home/administrateur/Bureau/frise_des_parcelles_corr.csv', index=False)

In [ ]:
ndephy = sdc.merge(dispositif, on='dispositif_id', how='left')
ndephy = ndephy.merge(domaine, on='domaine_id', how='left')

pattern = r'^[A-Z]{3}(?:[0-9]|X)\d{4}$'
mask_vide = ((ndephy['code_dephy'].isnull()) | (ndephy['code_dephy']=='NAN') | (ndephy['code_dephy']=='-')  | (ndephy['code_dephy']=='9999'))

df_dephy_ndephy_vide = ndephy.loc[(ndephy['dispositif_type']!='NOT_DEPHY') & (mask_vide)]
df_dephy_ndephy_non_vide = ndephy.loc[(ndephy['dispositif_type']!='NOT_DEPHY') & (~mask_vide)]
df_invalid_num_dephy = df_dephy_ndephy_non_vide[~df_dephy_ndephy_non_vide['code_dephy'].str.fullmatch(pattern, na=False)]

df_invalid_num_dephy_apres_nettoyage = df_dephy_ndephy_non_vide
df_invalid_num_dephy_apres_nettoyage['code_dephy'] = df_invalid_num_dephy_apres_nettoyage['code_dephy'].astype(str).\
    str.replace('PPZ', '').\
        str.replace('_', '').\
            str.replace(' ', '').\
                str.replace('\t', '').\
                    str.upper()

print(len(ndephy))
print(len(df_dephy_ndephy_vide))
print(len(df_dephy_ndephy_non_vide))
print(len(df_invalid_num_dephy))
print(len(df_invalid_num_dephy_apres_nettoyage))

37595
6141
29431
700
29431


/tmp/ipykernel_333004/484735412.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_invalid_num_dephy_apres_nettoyage['code_dephy'] = df_invalid_num_dephy_apres_nettoyage['code_dephy'].astype(str).\


KeyError: 'True: boolean label can not be used without a boolean index'